# Libraries

In [1]:
from GPTmuseVAE import GPTmuseVAE

import os
import torch
from GPTmuseVAE import GPTmuseVAE
from miditok.pytorch_data import DatasetTok
from miditok import REMI
from torchtoolkit.data import create_subsets
from pathlib import Path
from utils import *
import pygame
from pygame import mixer
import random

/Users/sebastianballesteros/Desktop/ie_classes/Semester_7/Advanced_AI/group_project_final/GPfinalvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Hyperparameters

In [2]:
# Model Hyperparameters
n_embd = 64
n_head = 8
n_layer = 4
z_dim = 16
block_size = 254 # what is the maximum context length for predictions?
dropout = 0.2
########################

# Hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# ------------

# Load Tokenized Dataset

In [3]:
tokenizer = REMI(params= Path('midi_dataset_tokenizer_bpe.conf'))
vocab_size = len(tokenizer)

tokens_paths = list(Path('midi_dataset_tokens_no_bpe').glob("**/*.json"))

dataset = DatasetTok(
    tokens_paths, 
    max_seq_len=block_size, # to make target and prediction match the song length of block size
    min_seq_len=block_size, 
    one_token_stream= False,
    func_to_get_labels = get_artist_label
)

Loading data: midi_dataset_tokens_no_bpe/midi_metal/Slayer:   0%|          | 0/511 [00:00<?, ?it/s]

Loading data: midi_dataset_tokens_no_bpe/midi_metal/Slayer: 100%|██████████| 511/511 [00:01<00:00, 259.20it/s]


# Load Model

In [4]:
model = GPTmuseVAE( vocab_size= len(tokenizer),
                    n_embd = n_embd,
                    n_head = n_head,
                    n_layer = n_layer,
                    z_dim = z_dim,
                    block_size = block_size,
                    dropout = dropout)


m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')



0.481936 M parameters


In [5]:
loaded_state_dict = torch.load('checkpoints/checkpoint_6500.pt')
model.load_state_dict(loaded_state_dict['model_state_dict'])

<All keys matched successfully>

# Find Pointers

In [6]:
_ , small_data = create_subsets(dataset, [0.01])
z, labels = process_dataset_for_z(small_data)
z = model.sample_latent(z)
pointer_dict = calculate_feature_pointers(z,labels)

# Generate

In [7]:
get_artist_label.artist_id_mapping

{'Slayer': 0,
 'Judas Priest': 1,
 'Black_sabath': 2,
 'Pantera': 3,
 'Ozzy Osbourne': 4,
 'Sepultura': 5,
 'Children Of Bodom': 6,
 'Carcass': 7,
 'Megadeth': 8,
 'Type O Negative': 9,
 'midi_pop_songs': 10,
 'Mozart': 11,
 'Ravel': 12,
 'Dvorak': 13,
 'Beethoven': 14,
 'Haydn': 15,
 'Schubert': 16,
 'Cambini': 17,
 'Bach': 18,
 'Brahms': 19,
 'Faure': 20}

In [25]:
aritst = 14
positions = torch.where(dataset[:]['labels'] == aritst)[0]
pointer = pointer_dict['Slayer']
magnitude = 20
#pointer = None

# Convert to a list for random sampling
positions_list = positions.tolist()

song_flag = random.sample(positions_list, 1)[0]

print(song_flag)

input_block_size = block_size

max_new_tokens = 64

gen_seed = dataset[song_flag]['input_ids'].unsqueeze(0)
generated_sequence = model.generate(gen_seed[:input_block_size] ,max_new_tokens=max_new_tokens, latent_vector = pointer, magnitude = magnitude)
out = generated_sequence[0].cpu().numpy().tolist()
gen_midi = tokenizer.tokens_to_midi(out)
gen_midi.dump('musicGPT_latent.mid')
mixer.init()
mixer.music.load("musicGPT_latent.mid")
mixer.music.play()

16655


In [11]:
mixer.music.play()

In [10]:
mixer.music.stop()